In [1]:
import tensorflow as tf
from tfx import v1 as tfx

print(tf.__version__)
print(tfx.__version__)

import os
import urllib.request
import tempfile

ImportError: DLL load failed while importing _ssl: The specified module could not be found.